In [1]:
import os
from google.colab import drive
drive.mount('/content/drive')
os.chdir('/content/drive/MyDrive/CML_2_Projects/Project 2/')

ModuleNotFoundError: No module named 'google'

In [2]:
from utils import helper_functions
import pandas as pd
import seaborn as sns
import numpy as np
import sklearn
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score,recall_score,precision_score,f1_score, confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_predict as cvp
from sklearn.impute import SimpleImputer
from sklearn.metrics import make_scorer

ModuleNotFoundError: No module named 'helper_functions'

In [ ]:
train_data = pd.read_csv('mimic_train.csv')
test_data = pd.read_csv('mimic_test_death.csv')

In [ ]:
#assign features
features_to_drop = ['DOD', 'DISCHTIME', 'DEATHTIME', 'LOS']
identifiers = ['subject_id', 'hadm_id', 'icustay_id']
numerical_features = ['HeartRate_Min', 'HeartRate_Max', 'HeartRate_Mean', 'SysBP_Min',
       'SysBP_Max', 'SysBP_Mean', 'DiasBP_Min', 'DiasBP_Max', 'DiasBP_Mean',
       'MeanBP_Min', 'MeanBP_Max', 'MeanBP_Mean', 'RespRate_Min',
       'RespRate_Max', 'RespRate_Mean', 'TempC_Min', 'TempC_Max', 'TempC_Mean',
       'SpO2_Min', 'SpO2_Max', 'SpO2_Mean', 'Glucose_Min', 'Glucose_Max',
       'Glucose_Mean']
categorical_features = ['GENDER', 'DOB', 'ADMITTIME','Diff', 'ADMISSION_TYPE', 'INSURANCE', 'RELIGION',
       'MARITAL_STATUS', 'ETHNICITY', 'DIAGNOSIS', 'ICD9_diagnosis',
       'FIRST_CAREUNIT']
target = ['HOSPITAL_EXPIRE_FLAG']

In [ ]:
#impute for numerical features
numerical_features
imp_num = SimpleImputer(strategy="mean")
imp_num.fit(X[numerical_features])
train_data[numerical_features] = imp_num.transform(train_data[numerical_features])
test_data[numerical_features] = imp_num.transform(test_data[numerical_features])

In [ ]:
#impute for categorical features
categorical_features
imp_cat = SimpleImputer(strategy="mode")
imp_cat.fit(X[categorical_features])
train_data[categorical_features] = imp_cat.transform(train_data[categorical_features])
test_data[categorical_features] = imp_cat.transform(test_data[categorical_features])

In [ ]:
#scale features (only numerical features)
scaler = StandardScaler()
scaler.fit(train_data[numerical_features])
train_data[numerical_features] = scaler.transform(train_data[numerical_features])
test_data[numerical_features] = scaler.transform(test_data[numerical_features])

In [ ]:
# first simple linear implementation
from sklearn.metrics import make_scorer
linear_svc = SVC(kernel='linear', probability = True) #class_weight imbalance not addressed 
grid_values = {'C':[0.1, 1, 10]}
#auc_scorer = make_scorer(roc_auc_score, needs_proba=True)
grid_linear_svc = GridSearchCV(linear_svc, param_grid = grid_values,scoring = 'roc_auc', cv=5)
grid_linear_svc.fit(train_data[numerical_features], train_data[target])
y_hat = grid_linear_svc.predict_proba(train_data[numerical_features]) 
print(grid_linear_svc.best_estimator_)

In [ ]:
class_labels = ["Survived","Died"]
labels = [0,1]
cm = confusion_matrix(y_pred=y_hat, y_true=train_data[target], labels=labels)
plot_confusion_matrix(cm, class_labels)

In [ ]:
#get_auc(train_data[target], y_hat, class_labels, column=1, plot=True)
#roc_auc_score(train_data_simple_scaled_skew['Cover_Type'], train_data_simple_scaled_skew_balanced_probs)

In [ ]:
# first simple non-linear implementation 
## then use kernel
rbf_svc = SVC(kernel='rbf', probability = True)
grid_values = {'C':[0.01, 0.1, 1, 10, 100], 'gamma':[0.01, 0.1, 0.2, 0.4, 0.6, 0.8, 0.99]} #will try more values for gamma in extended project
# but it is super stupid to optimize for accuracy 
grid_rbf_svc = GridSearchCV(rbf_svc, param_grid=grid_values, scoring = 'roc_auc', cv = 5)
grid_rbf_svc.fit(train_data[numerical_features], train_data[target])
y_hat = grid_rbf_svc.predict_proba(train_data[numerical_features])
print(classification_report(train_data[target], y_hat))
print(grid_rbf_svc.best_estimator_)

In [ ]:
#get_auc(train_data[target], y_hat, class_labels, column=1, plot=True)
#roc_auc_score(train_data[target], y_hat)

In [ ]:
cm = confusion_matrix(y_pred=y_hat, y_true=train_data[target], labels=labels)
plot_confusion_matrix(cm, class_labels)

## Kaggle

In [ ]:
# Produce .csv for kaggle testing 
y_hat_test = grid_rbf_svc.predict_proba(test_data[numerical_features])
test_predictions_submit = pd.DataFrame({"icustay_id": test_data["icustay_id"], "HOSPITAL_EXPIRE_FLAG": y_hat_test})
test_predictions_submit.to_csv("test_predictions_submit.csv", index = False)